<a href="https://colab.research.google.com/github/Xujjjjun2002/AI/blob/main/GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from d2l import torch as d2l

batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

In [ ]:
## 初始化参数
def get_params(vocab_size,num_hiddens,device):
  num_imputs = num_outs = vocab_size

  def normal(shape):
    return torch.randn(size=shape , device=device)*0.01 ## 均值为0，方差为0.01^2的随机数

  def three():
    return (normal((num_inputs, num_hiddens)),
            normal((num_inputs, num_hiddens)),
            torch.zeros(num_inputs, num_hiddens)) ##为什么不直接采用0作为初始化，而要用很小很小的随机数

  W_xz , W_hz , b_z = three()
  W_xr , W_hr , b_r = three()
  W_xh , W_hh , b_h = three()

  #输出层
  W_hq = normal((num_hiddens , num_outputs))
  b_q = torch.zeros(num_outputs , device=device)

  #附加梯度
  params = [W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q]
  for param in params:
    param.requires_grad_(True)
  return params

In [ ]:
def init_gru_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

In [ ]:
def gru(inputs , state , params):
  W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
  H, = state
  outputs = []
  for X in inputs:
    Z = torch.sigmod((X @ W_xz) + (H @ W_hz) + b_z)
    R = torch.sigmod((X @ W_xr) + (H @ W_hr) + b_r)
    H_tilda = torch.tanh((x @ W_xh) + ((R * H) @ W_hh) + b_h)
    H = Z * H + (1 - Z) * H_tilda
    Y = H @ W_hq + b_q
    outputs.append(Y)
  return torch.cat(outputs, dim=0), (H,)

In [ ]:
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        Z = torch.sigmoid((X @ W_xz) + (H @ W_hz) + b_z)
        R = torch.sigmoid((X @ W_xr) + (H @ W_hr) + b_r)
        H_tilda = torch.tanh((X @ W_xh) + ((R * H) @ W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = H @ W_hq + b_q
        outputs.append(Y)
    return torch.cat(outputs, dim=0), (H,)

In [ ]:
from google.colab import files
uploaded = files.upload()